In [39]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [40]:
df = pd.read_csv('data/test.csv', index_col='PassengerId')
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [41]:
df.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

## Pre-processing Task 1

In [42]:
# Objective : Adding Title column, extracting Title information from Name column
from title import title
# df.loc[df.Name.apply(title).isnull()].Name
df['Title'] = df.Name.apply(title)
df.Title.value_counts()

Mr.        240
Miss.       78
Mrs.        72
Master.     21
Rev.         2
Col.         2
Ms.          1
Dr.          1
Dona.        1
Name: Title, dtype: int64

In [43]:
# Importing the previously persisted pickle (the pandaa Series object, in this case here)
# age_map = pd.read_pickle("pickle/age_map.pkl")
# age_map = pd.read_csv('data/age_map.csv', index_col='Title')
# age_map = age_map['Age']
# age_map

In [44]:
# Next  step requires installation of this package
# !pip install ipynb

In [45]:
# Importing the function definition from previously defined notebook (one_eda)

# from ipynb.fs.defs.one_eda import guess_age
# from ipynb.fs.full.one_eda import guess_age
#test import
# guess_age(df.iloc[1,:])

# Alternate Solution that works but is really bad practicec as this can pollute the current namespace with variables from other
# %%capture
# %run one_eda.ipynb
# guess_age(df.iloc[1,:])

In [46]:
# Impute missing Ages
from guess_age import guess_age
# guess_age(df.iloc[1,:]) # For testing purpose to see if the function definition is loaded and works fine
df.loc[df.Age.isnull()] = df.loc[df.Age.isnull()].apply(guess_age, axis=1)
df.isnull().sum()

Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
Title         0
dtype: int64

In [47]:
# Objective Missing Fare
df.loc[df.Fare.isnull()] 

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,Mr.


In [48]:
from age_group import age_group
df['AgeGroup'] = df.Age.map(age_group)
df['AgeGroup'].head()

PassengerId
892    adult
893    adult
894     aged
895    adult
896    adult
Name: AgeGroup, dtype: object

In [49]:
fare_pivot_table = pd.read_pickle('pickle/fare_pivot_table.pkl')
fare_pivot_table

Fare  RoundedFare
Embarked Pclass AgeGroup                         
C        1      adult     104.559530   105.068493
                aged       60.117520    60.000000
                teen      138.234529   138.571429
         2      adult      23.879808    23.076923
                baby       37.004200    40.000000
                child      41.579200    40.000000
                teen       21.035400    20.000000
         3      adult      10.237776    11.521739
                baby       15.693750    17.500000
                child      16.199983    18.333333
                teen       10.921690    10.000000
Q        1      adult      90.000000    90.000000
         2      adult      12.350000    10.000000
         3      adult      10.471877    12.500000
                aged        7.750000    10.000000
                baby       29.125000    30.000000
                child      29.125000    30.000000
                teen        7.481950    10.000000
S        1      adult      67.040090    67.378641
                aged       59.998258    60.833333
                baby      151.550000   150.000000
                child     100.929150   100.000000
                teen       96.627080    97.000000
         2      adult      19.522868    18.992248
                aged       17.625000    17.500000
                baby       25.541667    26.666667
                child      28.527778    30.000000
                teen       20.921875    20.625000
         3      adult      13.211843    14.813433
                aged        7.866667    10.000000
                baby       26.098330    26.000000
                child      28.121296    28.148148
                teen       12.993980    14.444444

In [50]:
df.loc[df.Fare.isnull()] # Know their following details : Embarked, Pclass, AgeGroup to guess their fare

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,AgeGroup
PassengerId,,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,Mr.,aged


In [51]:
impute_fare_value = fare_pivot_table.loc[('S',3,'aged'),['RoundedFare']].values[0]

In [52]:
df.loc[df.Fare.isnull(),'Fare'] = impute_fare_value
df.loc[df.Fare.isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,AgeGroup
PassengerId,,,,,,,,,,,,


In [53]:
df.isnull().sum()

Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       327
Embarked      0
Title         0
AgeGroup      0
dtype: int64

## Removing Meta-data columns that are not Features and Duplicate Features

In [54]:
df.drop(columns=['Cabin','Name', 'Ticket'], inplace=True)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeGroup
PassengerId,,,,,,,,,
892,3,male,34.5,0,0,7.8292,Q,Mr.,adult
893,3,female,47.0,1,0,7.0000,S,Mrs.,adult
894,2,male,62.0,0,0,9.6875,Q,Mr.,aged
895,3,male,27.0,0,0,8.6625,S,Mr.,adult
896,3,female,22.0,1,1,12.2875,S,Mrs.,adult


In [55]:
from rounded_fare import rounded_fare
df['RoundedFare'] = df.Fare.map(rounded_fare)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeGroup,RoundedFare
PassengerId,,,,,,,,,,
892,3,male,34.5,0,0,7.8292,Q,Mr.,adult,10
893,3,female,47.0,1,0,7.0000,S,Mrs.,adult,10
894,2,male,62.0,0,0,9.6875,Q,Mr.,aged,10
895,3,male,27.0,0,0,8.6625,S,Mr.,adult,10
896,3,female,22.0,1,1,12.2875,S,Mrs.,adult,10


In [56]:
# Because Age feature is transformed to AgeGroup, Age can be removed?? No, remove AgeGroup. Age works better for modelling. AgeGroup has too many categories.
# Bacause Fare feature is transformed to RoundedFare, Fare can be removed?
# Because Title is not helpful for prediction
df.drop(columns=['AgeGroup','Fare', 'Title'], inplace=True)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked,RoundedFare
PassengerId,,,,,,,
892,3,male,34.5,0,0,Q,10
893,3,female,47.0,1,0,S,10
894,2,male,62.0,0,0,Q,10
895,3,male,27.0,0,0,S,10
896,3,female,22.0,1,1,S,10


## Final Task : Persist processed test-dataset

In [57]:
df.to_csv('data/test_processed_1.csv')